### Etapa de gerar os dados geomorfológicos
##### Variáveis utilizadas e bibliografia

- 

In [ ]:
%pip install rioxarray xarray numpy scipy rasterio

In [8]:
# Libs utilizadas
import rioxarray as rxr
import xarray as xr
import numpy as np
from pyproj import CRS
from rasterio.warp import transform
from math import floor
from scipy.ndimage import convolve

In [5]:
mde_path = r"d:\Mestrado\Trabalho Final\SIG\MDT.tif"
uso_solo_path = r"d:\Mestrado\Trabalho Final\SIG\USOSOLO.tif"

uso_solo = rxr.open_rasterio(uso_solo_path)
mde = rxr.open_rasterio(mde_path)

In [ ]:
dem = mde.squeeze(drop=True)  # (y,x), float
Z = dem.data.astype(float)
nodata = np.isnan(Z)

# resoluções (m) assumindo grid regular
dx = float(abs(dem.x.diff("x").mean()))
dy = float(abs(dem.y.diff("y").mean()))

# preenche NaN temporariamente para não poluir convoluções
Z_filled = Z.copy()
if np.any(nodata):
    Z_filled[nodata] = np.nanmean(Z_filled)


In [ ]:
# --- Kernels de Horn (1ª ordem e 2ª ordem) ---
Kx  = np.array([[-1,0,1],[-2,0,2],[-1,0,1]]) / (8*dx)
Ky  = np.array([[-1,-2,-1],[0,0,0],[1,2,1]]) / (8*dy)
Kxx = np.array([[1,-2,1],[2,-4,2],[1,-2,1]]) / (3*dx*dx)
Kyy = np.array([[1,2,1],[-2,-4,-2],[1,2,1]]) / (3*dy*dy)
Kxy = np.array([[-1,0,1],[0,0,0],[1,0,-1]]) / (4*dx*dy)

# derivadas
p = convolve(Z_filled, Kx,  mode="nearest")
q = convolve(Z_filled, Ky,  mode="nearest")
r = convolve(Z_filled, Kxx, mode="nearest")
t = convolve(Z_filled, Kyy, mode="nearest")
s = convolve(Z_filled, Kxy, mode="nearest")

for arr in (p,q,r,s,t):
    arr[nodata] = np.nan

# --- Métricas ---
eps = 1e-12
grad = np.sqrt(p**2 + q**2)

# declividade
slope_rad = np.arctan(grad)
slope_deg = np.degrees(slope_rad)

# aspecto
aspect = np.degrees(np.arctan2(q, -p))
aspect = np.where(aspect < 0, 360 + aspect, aspect)

# curvaturas
den = (grad**3) + eps
curv_profile = (r*(p**2) + 2*s*p*q + t*(q**2)) / den
curv_plan    = (r*(q**2) - 2*s*p*q + t*(p**2)) / den
curv_gauss   = (r*t - s**2) / ((1 + p**2 + q**2)**2 + eps)

# TPI e TRI (janela 3x3)
K8 = np.array([[1,1,1],[1,0,1],[1,1,1]], float)
K8 /= K8.sum()
neighbor_mean = convolve(Z_filled, K8, mode="nearest")

tpi = Z_filled - neighbor_mean
tpi[nodata] = np.nan

abs_diff = np.abs(Z_filled - neighbor_mean)
tri = convolve(abs_diff, K8, mode="nearest")
tri[nodata] = np.nan


In [ ]:
# lista de variáveis
band_list = [
    ("elevation",    Z.astype(np.float32)),
    ("slope_deg",    slope_deg.astype(np.float32)),
    ("aspect_deg",   aspect.astype(np.float32)),
    ("curv_profile", curv_profile.astype(np.float32)),
    ("curv_plan",    curv_plan.astype(np.float32)),
    ("curv_gauss",   curv_gauss.astype(np.float32)),
    ("tpi",          tpi.astype(np.float32)),
    ("tri",          tri.astype(np.float32)),
]

# criar stack multibanda
stack = xr.concat(
    [xr.DataArray(arr, dims=("y","x"), name=name) for name,arr in band_list],
    dim="band"
).assign_coords(band=np.arange(1, len(band_list)+1))

# herdar CRS/transform do MDT original
stack = stack.rio.write_crs(dem.rio.crs)
stack = stack.rio.write_transform(dem.rio.transform())

# nomes das bandas
stack.attrs["band_names"] = [name for name,_ in band_list]

# salvar
out_path = "mdt_variaveis_morfologicas.tif"
stack.rio.to_raster(out_path, compress="deflate", BIGTIFF="IF_SAFER")

print("Arquivo salvo:", out_path)
print("Bandas:", stack.attrs["band_names"])
